In [7]:
import urllib.request
from bs4 import BeautifulSoup as Soup
import datetime
from dateutil import tz
from time import sleep
from typing import List
import pandas as pd

In [34]:
import urllib.request
from bs4 import BeautifulSoup as Soup
import datetime
from dateutil import tz
from time import sleep
from typing import List
### CLASSEs
class GoogleNews:

    def __init__(self, account:str, password:str, proxy_list=List[str],lang="en", period="", encode="utf-8", region=None):
        self.account = account
        self.password = password
        self.proxy_list = proxy_list
        self.user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:64.0) Gecko/20100101 Firefox/64.0'
        self.__lang = lang
        # 若user提供region，則headers加入accept-language，否則不加
        if region:
            self.accept_language= lang + '-' + region + ',' + lang + ';q=0.9'
            self.headers = {'User-Agent': self.user_agent, 'Accept-Language': self.accept_language}
        else:
            self.headers = {'User-Agent': self.user_agent}
        self.__period = period
        self.__encode = encode
        self.proxy = self.proxy_check(proxy_list=self.proxy_list)
        self.__results = self.result_df_init()

    def response_builder(self, url, proxy, is_google_news_search=False):
        """
        將proxy設置於opener，並回傳req
        is_google_news_search: 搜尋goolge news時要設為True
        """
        if is_google_news_search:
            self.req = urllib.request.Request(self.url.replace("search?","search?hl="+self.__lang+"&gl="+self.__lang+"&"), headers=self.headers)
        else:
            self.req = urllib.request.Request(url, headers=self.headers)

        self.proxy = proxy
        self.proxy_support = urllib.request.ProxyHandler(
                            {
                                'http':f'http://{self.account}:{self.password}@{proxy}', 
                                'https':f'https://{self.account}:{self.password}@{proxy}',
                            })

        opener = urllib.request.build_opener(self.proxy_support)
        urllib.request.install_opener(opener)
        self.response = urllib.request.urlopen(self.req, timeout=10)

        return self.response

    def proxy_check(self, proxy_list):
        """
        有時公司個別proxy會失效
        讀取所有的proxy，嘗試有用的proxy，並回傳第一個有用的proxy
        """
        self.work_proxies = []
        test_url = "https://www.google.com/?hl=zh_TW"

        retries = 3 # 每個proxy_list中的proxy嘗試次數
        print('proxy check')
        for proxy in proxy_list:
            print('proxy= ', proxy, end='')
            for attempt in range(retries):
                if self.work_proxies==[]:
                    try:
                        response = self.response_builder(url=test_url, proxy=proxy, is_google_news_search=False)
                        # 成功連線
                        response.close()
                        self.work_proxies.append(proxy)
                        print(" ok")
                    except Exception as e:
                        print(f' failed (Exception: {e}')
                        sleep(3)

        if self.work_proxies == []: 
            print("目前並無有效proxy")
        else:
            self.proxy = self.work_proxies[0]
            return self.proxy

    def utc_to_taipei(self, date: str):
        
        try:
            utc = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
            from_zone = tz.gettz('UTC')
            to_zone = tz.gettz('Asia/Taipei')
            # Tell the datetime object that it's in UTC time zone since 
            # datetime objects are 'naive' by default
            utc = utc.replace(tzinfo=from_zone)
            taipei = utc.astimezone(to_zone)
            taipei_strtime = taipei.strftime("%Y-%m-%d %H:%M:%S")
            return taipei_strtime
        except Exception as e:
            print(e)
            return None
        
    def result_df_init(self):
        results_col = ['title', 'desc', 'date', 'datetime', 'link', 'img', 'media', 'site', 'keyword', 'category']
        self.__results = pd.DataFrame(columns=results_col)
        return self.__results

    def get_news(self, key="", key_catogory="", deamplify=False):
        
        my_key = key

        if self.__encode != "":
            key = urllib.request.quote(key.encode(self.__encode))

        if key == "":
            self.url = 'https://news.google.com/?hl={}'.format(self.__lang)
        else:
            key = "+".join(key.split(" ")) 
            self.url = 'https://news.google.com/search?q={}+when:{}&hl={}'.format( key, self.__period, self.__lang.lower())

    
        retries = 3 # 嘗試連線次數
        for attempt in range(retries):
            try:
                self.response = self.response_builder(url=self.url, proxy=self.proxy, is_google_news_search=True)
                self.page = self.response.read()
                self.content = Soup(self.page, "html.parser")
                self.response.close()    
                break
            except Exception as e:  # 連線錯誤，articles尚未迭帶
                print(e)
                print(f"搜尋時連線錯誤: 關鍵字-{my_key}(等待10秒後重試)")
                sleep(10)
                self.response = None
                self.proxy = self.proxy_check(proxy_list=self.proxy_list)
                

        if self.response is None:
            print(f"搜尋時連線錯誤: 停止搜關鍵字-{my_key}")
            return
        
        elif self.response is not None:           
            # 開始解析文章
            articles = self.content.select('div[class="NiLAwe y6IFtc R7GTQ keNKEd j7vNaf nID9nc"]')
            for article in articles:
                try:

                    # title
                    try:
                        title=article.find('h3').text
                    except:
                        title=None

                    # description
                    try:
                        desc=article.find('span').text
                    except:
                        desc=None

                    # date
                    try:
                        date = article.find("time").text
                    except:
                        date = None

                    # datetime
                    try:
                        datetime=article.find('time').get('datetime')
                        datetime_chars=self.utc_to_taipei(datetime)
                    except:
                        datetime_chars=None 

                    # link
                    if not deamplify:
                        link = 'news.google.com/' + article.find("h3").find("a").get("href")
                    else:
                        try:
                            link = 'news.google.com/' + article.find("h3").find("a").get("href")
                        except Exception as deamp_e:
                            print(deamp_e)
                            link = article.find("article").get("jslog").split('2:')[1].split(';')[0]

                    if link.startswith('https://www.youtube.com/watch?v='):
                        desc = 'video'

                    # image
                    try:
                        img = article.find("img").get("src")
                    except:
                        img = None

                    # site
                    try:
                        site=article.select('a[class*="wEwyrc"]')[0].text
                    except:
                        site=None

                    # collection
                    data = [title, desc, date, datetime_chars, link, img, None, site, my_key, key_catogory]
                    self.__results.loc[len(self.__results)] = data
                
                    
                # 解析錯誤
                except Exception as e_article:                   
                    print(e_article, f"{my_key}一篇文章解析錯誤")
                    return None

            return self.__results
        
    def results(self,sort=False):
        """Returns the __results.
        New feature: include datatime and sort the articles in decreasing order"""
        results=self.__results
        if sort:
            try:
                results=results.sort_values(by='datetime',ascending=False)
            except:
                print("No datetime column")
        return results


In [4]:
import pandas as pd
from tqdm import tqdm
from google_news_crawl import GoogleNews
import pyshorteners
import requests
import hanzidentifier
from time import sleep
import datetime

### 分詞用 ###
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger

### 讀取模型用 ###
import dill

class crawler_data_getter:

    ### set parameters
    # 建立爬蟲物件: 連到外網需要proxy才能連，而proxy中要打入nt帳密
    proxy_list = pd.read_excel("./crawler_config/proxy.xlsx", header=None)[0].tolist()
    nt_account = pd.read_excel("./crawler_config/account.xlsx")
    auo_account = nt_account["NTaccount"][0]
    auo_password = nt_account["password"][0]
    period = '1d'
    googlenews = GoogleNews(lang='zh-TW', period=period, account=auo_account, password=auo_password, proxy_list=proxy_list)
    # 搜尋詞列表: 爬蟲會先用預設好的關鍵字列表去搜尋新聞，並在後續會依照關鍵字所屬的類別來分類新聞
    key_words_df = pd.read_excel("./crawler_config/keyword.xlsx")
    # 停用標題列表
    drop_list_path = "./crawler_config/dropword.xlsx"
    drop_series = pd.read_excel(drop_list_path, header=None)[0]
    # 出處列表
    site_list_path = "./crawler_config/NewsSite.xlsx"
    site_series = pd.read_excel(site_list_path)["出處"].str.replace(" ","").str.lower()

    # 短網址用
    link_col_name = "link"

    # 分詞用
    # ckip分詞模型父資料夾
    ckip_path = ".\\ckiplab\\"
    ws_model = 'albert-base-chinese-ws'
    pos_model = 'albert-base-chinese-pos'
    title_col_name = "title"

    # 爬蟲用
    keyword_col_name = "keyword" #關鍵字欄位名
    category_col_name = "category" #關鍵字類別欄位名

    # 預測用
    # 預測變數名稱
    variable_names = ["click", "DailyNews"]
    # 模型路徑
    model_path = "./models/"
    # 從爬蟲表格選的部分欄位
    selected_col = ['title', 'datetime', 'link', 'short_link', 'site', 'category']

    ### 開始爬蟲
    def start_crawl(self):
        """
        用爬蟲物件做爬資料的動作
        """
        tqdm.pandas(desc="用所有關鍵字去爬取一天內的Google News，並紀錄標題、描述、日期...等資訊")
        search_action = self.key_words_df.progress_apply(lambda row: self.googlenews.get_news(key=row[self.keyword_col_name], key_catogory=row[self.category_col_name]), axis=1)

    def get_crawl_result(self):
        """
        爬蟲結束後，取出爬到的結果
        """
        self.search_result_df = self.googlenews.results()

    def crawl_data_cleaner(self):
        """
        篩選資料
        """
        # 篩選掉有停用的標題、出處的資料
        print("篩選掉不可用的標題、不可用的新聞出處")
        
        def title_site_filter(row:pd.Series, drop_series:pd.Series, site_series:pd.Series, title_name:str="title", site_name:str="site"):
            """
            row: 搜尋結果的一列
            drop_series: 停用標題列表
            site_series: 出處列表
            title_name: 標題欄位名
            site_name: 出處欄位名
            """
            title  = row[title_name]
            site = row[site_name]

            # 只保留含有關鍵字的標題
            condition_1 = (hanzidentifier.is_traditional(title))
            
            # 只保留標題是繁體中文，或標題含有'彭双浪'的文章
            condition_2 = ("彭双浪" in title) 

            # 只保留標題內不含dropword.xlsx指定文字的文章
            condition_3 = drop_series.apply(lambda drop_word: drop_word not in title).all()

            # 只保留來源自指定的新聞出處的文章
            condition_4 = site in site_series.to_list()

            return ( condition_1 or condition_2) and condition_3 and condition_4

        row_filter = self.search_result_df.apply(lambda row: title_site_filter( row, drop_series=self.drop_series, site_series=self.site_series), axis=1)
        self.search_result_df = self.search_result_df[row_filter]

        # 排除link欄位重複的資料，保留第一筆
        print("篩選掉重複資料")
        self.search_result_df.drop_duplicates(subset='link', keep="first", inplace=True)

        # 排除title、datetime、site三個欄位為整體，都重複才排除
        # search_result_df.drop_duplicates(subset=['title','datetime','site'], keep='first', inplace=True)

        # 排除title重複
        self.search_result_df.drop_duplicates(subset=['title'], keep='first', inplace=True)

        # 依照catogory來排序
        print("依新聞類別排序資料")
        self.search_result_df = (self.search_result_df
                                    .assign(**{"sort": lambda search_result_df: self.search_result_df[self.category_col_name] # 新增sort欄位，sort的值依照catogory來排序編號
                                                                                .apply(lambda catogory: {'產業':0, '法令':1, 'HR':2}[catogory] if catogory in {'產業', '法令', 'HR'} else 3) 
                                        }
                                    )
                                    .sort_values(["sort"]) # 依照sort欄位的數值大小來排序
                                    .drop(columns=["sort"])) # 去除sort欄位

    def news_link_shorten(self):
        """
        將爬蟲結果中的新聞連結縮短網址
        """
        ### 縮短爬蟲資料網址
        def url_shorten(auo_account:str, auo_password:str, proxy:str, url:str):
            """
            apply用
            auo_account: auo帳號
            auo_password: auo密碼
            proxy: proxy
            url: 要縮短的網址
            傳入要縮短的網址，回傳短網址
            """

            # 若url不是以"http://"或"https://"開頭，則加上"http://"
            if not url.startswith(("http://", "https://")):
                url = f"http://{url}"

            # 加入proxy
            session = pyshorteners.Shortener(
                proxies={
                    'http':f'http://{auo_account}:{auo_password}@{proxy}',
                    'https':f'http://{auo_account}:{auo_password}@{proxy}',
                },
                timeout=10 # 設置超時時間為10秒
            )

            # 縮短網址: 若連線超過10秒未回應，則等待10秒後重試，最多重試三次
            while True:
                patience = 3
                try:
                    short_url = session.tinyurl.short(url)
                    return short_url
                except requests.exceptions.Timeout:
                    print("錯誤: 短網址請求超過10秒未回應")
                    print("等待10秒後重試")
                    sleep(10)
                except requests.exceptions.RequestException as e:
                    print(f"發生連線錯誤: {e}")
                    print("等待10秒後重試")
                    sleep(10)
                except Exception:
                    print("縮短一個連結時發生連線外錯誤，等待10秒後重試")
                    sleep(10)
                    patience += 1 
                    if patience >= 3:
                        print("已重試三次，跳過此連結")
                        return None

        tqdm.pandas(desc="縮短所有Google News連結")
        self.search_result_df['short_link'] = self.search_result_df[self.link_col_name].progress_apply(lambda link: url_shorten(auo_account=self.auo_account, auo_password=self.auo_password, proxy=self.googlenews.proxy, url=link))

    def titles_tokenizer(self):
        """
        讀ckip模型對新聞標題做分詞並去除非名詞、非動詞
        返回一個處理過的字串串列
        """      
        
        # 建立分詞、詞性標記、命名實體物件
        ws_driver = CkipWordSegmenter(model_name=self.ckip_path + self.ws_model) # 分詞
        pos_driver = CkipPosTagger(model_name=self.ckip_path + self.pos_model)    # 詞性標記(POS)

        # 開始分詞、詞性標記，分詞結果放入ws，詞性標註結果放入pos        
        ws = ws_driver(self.search_result_df[self.title_col_name].tolist())
        pos = pos_driver(ws)

        print("去除非名詞、非動詞")
        # 用來保留主要詞彙的函式
        def clean(sentence_ws, sentence_pos): 

            short_sentence = []
            stop_pos = set(['Nep', 'Nh']) # 這 2種詞性不保留(指代定詞、代名詞)

            for word_ws, word_pos in zip(sentence_ws, sentence_pos):

                # 部分的分詞內有包含空白字元，會導致停用詞不會被辨識到，所以在這邊用replace處理掉空白字元
                word_ws = word_ws.replace(' ','')

                # 只留名詞和動詞
                is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N") or word_pos.startswith("FW")
                # 去掉名詞裡的某些詞性(指代定詞、代名詞、專有名詞)
                is_not_stop_pos = word_pos not in stop_pos
                # 只剩一個字的詞也不留
                is_not_one_charactor = not (len(word_ws) == 1)

                # 組成串列
                if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
                    short_sentence.append(f"{word_ws}")

            return " ".join(short_sentence)


        self.cleaned_title_list = pd.DataFrame({'ws':pd.Series(ws), 'pos':pd.Series(pos)}).apply(lambda row: clean(row['ws'], row['pos']), axis=1).tolist()


    def model_predict(self):
        """
        讀預測click、預測dailynews的模型，並對ckip處理後的標題做預測，並將預測機率、分類等，合併進原本的dataframe
        返回所需的最終表格
        """
        
        # 讀取模型
        pred_result_dic = dict()
        for name in self.variable_names:
            model_filename = f"model_{name}_best_f1.pkl"

            print(f"{name}模型預測")

            # 開啟模型
            with open( self.model_path + model_filename, "rb") as f:
                model = dill.load(f)

            # 提供預測結果及其機率值
            pred = model.predict(self.cleaned_title_list )
            prob = model.predict_proba(self.cleaned_title_list )[:,1]

            # 前面content_df的title在分詞前被處理過
            pred_result_dic[f'{name}_prob'] = prob
            pred_result_dic[f'{name}_prediction'] = pred

        pred_result = pd.DataFrame( pred_result_dic, columns=["click_prob", "DailyNews_prob", 'click_prediction', "DailyNews_prediction"]) 

        # 將預測結果合併進dataframe
        print("預測表格彙整")
        # 建立包含預測結果的pred_df
        pred_df = pd.concat([self.search_result_df[self.selected_col].reset_index(drop=True), pred_result], axis=1)

        # # 依照類別切出子表
        categories = pred_df.category.unique().tolist()
        sub_result_df_list = []
        for category in categories:
            sub_df_mask = pred_df.category == category
            sub_df = pred_df[sub_df_mask].copy()

            # 以機率值排序，讓機率高的優先排在前面
            sub_df["total_prob"] = sub_df["click_prob"] + sub_df["DailyNews_prob"]
            sub_df = sub_df.sort_values(by=["total_prob"], ascending=False)
            sub_df = sub_df.drop(columns=["total_prob"])

            # 彙整子表
            sub_result_df_list.append(sub_df)

        # # 合併子表
        self.pred_df = pd.concat(sub_result_df_list)

        # # # 寫出
        print("寫出預測表格")
        today = datetime.datetime.now().strftime("%Y%m%d")
        pred_df.to_excel(f".\\history_predict\\News_{today}_predict.xlsx", encoding='utf-8-sig', index=False)

    def main(self):
        self.start_crawl()
        self.get_crawl_result()
        ### 整理爬蟲資料
        print("開始整理爬蟲資料")
        self.crawl_data_cleaner()
        self.news_link_shorten()
        print("開始對標題分詞")
        self.titles_tokenizer()
        # ### **讀取模型**
        print("讀取預測模型") 
        self.model_predict()  
        print("爬蟲及預測完成")
        
        return self.pred_df



proxy check
proxy=  auhqproxy.cdn.corpnet.auo.com:8080 ok
proxy=  auhqwsg.corpnet.auo.com:8080proxy=  auhqproxy.corpnet.auo.com:8083

In [5]:
test_obj = crawler_data_getter()

In [6]:
test_obj.main()

用所有關鍵字去爬取一天內的Google News，並紀錄標題、描述、日期...等資訊:   2%|▏         | 2/82 [00:06<04:09,  3.12s/it]

The read operation timed out
proxy check
proxy=  auhqproxy.cdn.corpnet.auo.com:8080 failed (Exception: The read operation timed out
 ok
proxy=  auhqwsg.corpnet.auo.com:8080proxy=  auhqproxy.corpnet.auo.com:8083

用所有關鍵字去爬取一天內的Google News，並紀錄標題、描述、日期...等資訊:   5%|▍         | 4/82 [00:27<09:22,  7.21s/it]

<urlopen error timed out>
proxy check
proxy=  auhqproxy.cdn.corpnet.auo.com:8080 ok
proxy=  auhqwsg.corpnet.auo.com:8080proxy=  auhqproxy.corpnet.auo.com:8083

用所有關鍵字去爬取一天內的Google News，並紀錄標題、描述、日期...等資訊:   6%|▌         | 5/82 [00:33<08:33,  6.67s/it]

...

Inference: 100%|██████████| 3/3 [00:20<00:00,  6.76s/it]


去除非名詞、非動詞
讀取預測模型
click模型預測
DailyNews模型預測
預測表格彙整
寫出預測表格


C:\Users\BenBLLee\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


爬蟲及預測完成


,title,datetime,link,short_link,site,category,click_prob,DailyNews_prob,click_prediction,DailyNews_prediction
71,友達以上戀人未滿？ 六旬婦傳自拍修圖美照網戀差點遭詐,2023-11-02 09:10:09,news.google.com/./articles/CBMiJ2h0dHBzOi8vdWR...,https://tinyurl.com/ykwahjvw,聯合新聞網,產業,0.970760,0.879778,1,1
35,華邦電看半導體隧道透光Q4勝Q3明年再向上| 產經,2023-11-02 03:56:36,news.google.com/./articles/CBMiMGh0dHBzOi8vbmV...,https://tinyurl.com/yterkvua,非凡新聞,產業,0.686473,0.759857,1,1
49,全球首個漂浮式風漁融合發電完工可讓6千多家戶用一年,2023-11-01 11:51:00,news.google.com/./articles/CBMiMmh0dHBzOi8vd3d...,https://tinyurl.com/yw3ufqh6,工商時報,產業,0.733866,0.701341,1,1
38,日本半導體挾「2優勢」要抓回失去的40年！集邦：盤點未來可能形成 ...,2023-11-01 17:14:30,news.google.com/./articles/CBMiR2h0dHBzOi8vd3d...,https://tinyurl.com/yl3xesbk,財訊,產業,0.793341,0.641619,1,1
8,南科7個區都市計畫內政部審議通過 半導體生產完整基地,2023-11-01 19:16:35,news.google.com/./articles/CBMiLmh0dHBzOi8vbW9...,https://tinyurl.com/yvtfpjwa,經濟日報,產業,0.649125,0.766329,1,1
...,...,...,...,...,...,...,...,...,...,...
319,桃青年局首創2023 Taoyuan FuturePeak 桃園青創博覽會,2023-11-01 10:47:27,news.google.com/./articles/CBMiLmh0dHBzOi8vbW9...,https://tinyurl.com/ykre5tp9,經濟日報,HR,0.034195,0.015631,0,0
270,塑膠中心慶30助業界前瞻永續,2023-11-02 03:00:00,news.google.com/./articles/CBMiMmh0dHBzOi8vd3d...,https://tinyurl.com/ymyuefgv,工商時報,HR,0.025398,0.021787,0,0
317,學生打老師事件頻傳10縣市教師工會喊話教育部負起責任,2023-11-01 13:31:00,news.google.com/./articles/CBMiMmh0dHBzOi8vd3d...,https://tinyurl.com/yk8pptg6,工商時報,HR,0.024773,0.016306,0,0
364,矽品志工國、台、英三聲道錄製有聲書 帶領惠明盲童沉浸式體驗學習,2023-11-01 23:04:05,news.google.com/./articles/CBMiNmh0dHBzOi8vbmV...,https://tinyurl.com/ywyn356r,自由時報,HR,0.028163,0.007241,0,0
